In [7]:
import pandas as pd
import numpy as np
#df = pd.read_csv("‪C:/Users/asus/Desktop/DA_projects/karpov/band2/task2/bookings.csv")
df = pd.read_csv(r"https://stepik.org/media/attachments/lesson/360344/bookings.csv",sep=';')
df.head()

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   hotel                      119390 non-null  object 
 1   is_canceled                119390 non-null  int64  
 2   lead_time                  119390 non-null  int64  
 3   arrival_full_date          119390 non-null  object 
 4   arrival_date_year          119390 non-null  int64  
 5   arrival_date_month         119390 non-null  object 
 6   arrival_date_week_number   119390 non-null  int64  
 7   arrival_date_day_of_month  119390 non-null  int64  
 8   stays_in_weekend_nights    119390 non-null  int64  
 9   stays_in_week_nights       119390 non-null  int64  
 10  stays_total_nights         119390 non-null  int64  
 11  adults                     119390 non-null  int64  
 12  children                   119386 non-null  float64
 13  babies                     11

In [8]:
'''Приведите названия колонок к нижнему регистру и замените пробелы на знак нижнего подчеркивания.'''
df.columns = df.columns.str.lower().str.replace(' ','_')

In [27]:
'''Пользователи из каких стран совершили наибольшее число успешных бронирований? Укажите топ-5.'''
df[df['is_canceled']==0]['country'].value_counts()[:5]

PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: country, dtype: int64

In [72]:
'''На сколько ночей в среднем бронируют отели разных типов?'''
df.groupby('hotel',as_index=False).agg({'stays_total_nights':'mean'}).round(2)

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


In [73]:
'''Иногда тип номера, полученного клиентом (assigned_room_type),
отличается от изначально забронированного (reserved_room_type).Такое может произойти, например, 
по причине овербукинга.Сколько подобных наблюдений встретилось в датасете?'''
df[df['reserved_room_type']!=df['assigned_room_type']].shape[0]

14917

In [39]:
'''Проанализируйте даты запланированного прибытия. 
– На какой месяц чаще всего успешно оформляли бронь в 2016? 
Изменился ли самый популярный месяц в 2017?
– Сгруппируйте данные по годам и проверьте, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов'''
print('2016 :',df[df['arrival_date_year']==2016]['arrival_date_month'].value_counts()[:1])
print('2017 :',df[df['arrival_date_year']==2017]['arrival_date_month'].value_counts()[:1])

2016 : October    6203
Name: arrival_date_month, dtype: int64
2017 : May    6313
Name: arrival_date_month, dtype: int64


In [74]:
df[(df['is_canceled']==1) & (df['hotel']=='City Hotel')] \
.groupby(['arrival_date_year','arrival_date_month']).agg({'is_canceled':'count'}).idxmax()

is_canceled    (2017, May)
dtype: object

In [75]:
'''Посмотрите на числовые характеристики трёх переменных: adults, children и babies.
Какая из них имеет наибольшее среднее значение?'''
df[['adults','children','babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

In [59]:
'''Создайте колонку total_kids, объединив children и babies. 
Отели какого типа в среднем пользуются большей популярностью у клиентов с детьми?'''
df['total_kids'] = df['children']+df['babies']

In [62]:
df.groupby('hotel').agg({'total_kids':'mean'}).idxmax()

total_kids    Resort Hotel
dtype: object

In [64]:
'''Создайте переменную has_kids, которая принимает значение True, 
если клиент при бронировании указал хотя бы одного ребенка (total_kids), в противном случае – False.
Посчитайте отношение количества ушедших пользователей к общему количеству клиентов, 
выраженное в процентах (churn rate). Укажите, среди какой группы показатель выше.'''
df['has_kids'] = df['total_kids']!=0.0

'Создайте переменную has_kids, которая принимает значение True, \nесли клиент при бронировании указал хотя бы одного ребенка (total_kids), в противном случае – False.\nПосчитайте отношение количества ушедших пользователей к общему количеству клиентов, \nвыраженное в процентах (churn rate). Укажите, среди какой группы показатель выше.'

In [81]:
df[(df['has_kids']==False) & (df['is_canceled']==1)].shape[0]/df[df['has_kids']==False].shape[0]

0.3721900158104203